In [9]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from glob import glob

In [10]:
FEATURE_DIM = 74
LATENT_DIM = 100

In [11]:
class FlowDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        
        self.features = self.data_frame.drop(columns=["Label"])
        self.features = self.features.select_dtypes(include=[np.number])

        self.transform = transform

    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        features = self.features.iloc[idx].values.astype(np.float32)
        return torch.tensor(features)

In [12]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(LATENT_DIM, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, FEATURE_DIM),
            nn.Sigmoid()
        )
    
    def forward(self, z):
        return self.model(z)

In [13]:
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(FEATURE_DIM, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        return self.model(x)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

learning_rate = 0.0001

In [15]:
n_critic = 5
clip_value = 0.01
batch_size = 64
steps = 1e5

In [16]:
files = glob("./data/*.csv")

for file in files:
    basename = os.path.basename(file)
    basename = basename.replace(".csv", "").replace(" ", "_")

    dataset = FlowDataset(file)
    print(f"Training on {file}: {len(dataset)} samples")
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    generator = Generator().to(device)
    critic = Critic().to(device)
    critic_optimizer = optim.RMSprop(critic.parameters(), lr=learning_rate)
    generator_optimizer = optim.RMSprop(generator.parameters(), lr=learning_rate)

    epochs = int(steps / len(dataloader)) + 1

    for epoch in range(epochs):
        for i, real_data in enumerate(dataloader):
            real_data = real_data.to(device)
            batch_size = real_data.size(0)

            critic_optimizer.zero_grad()

            critic_real = critic(real_data)

            z = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_data = generator(z).detach()
            critic_fake = critic(fake_data)

            critic_loss = -(torch.mean(critic_real) - torch.mean(critic_fake))
            critic_loss.backward()
            critic_optimizer.step()

            for p in critic.parameters():
                p.data.clamp_(-clip_value, clip_value)
            
            if i % n_critic == 0:
                generator_optimizer.zero_grad()

                z = torch.randn(batch_size, LATENT_DIM, device=device)
                fake_data = generator(z)
                critic_fake = critic(fake_data)

                generator_loss = -torch.mean(critic_fake)
                generator_loss.backward()
                generator_optimizer.step()
        else:
            print(f"\rEpoch [{epoch:3d}/{epochs}] Batch {i+1}/{len(dataloader)} \
                Loss D: {critic_loss.item():.4f}, Loss G: {generator_loss.item():.4f}", end="")
        
        if epoch % 100 == 0:
            with torch.no_grad():
                z = torch.randn(64, LATENT_DIM, device=device)
                fake_data = generator(z).cpu().numpy()
                fake_data = pd.DataFrame(fake_data, columns=dataset.features.columns)
                fake_data.to_csv(f"./fake/fake_data_epoch_{epoch}.csv")
    else:
        torch.save({
            'generator_state_dict': generator.state_dict(),
            'critic_state_dict': critic.state_dict(),
            'generator_optimizer_state_dict': generator_optimizer.state_dict(),
            'critic_optimizer_state_dict': critic_optimizer.state_dict(),
        }, f'wgan_cicflowmeter_{basename}.pth')
        print()

print("\nTraining complete.")

Training on ./data/FTP-Patator.csv: 6878 samples
Epoch [925/926] Batch 108/108                 Loss D: -118911624.0000, Loss G: -0.1621
Training on ./data/PortScan.csv: 20000 samples
Epoch [319/320] Batch 313/313                 Loss D: -103156016.0000, Loss G: -0.1151
Training on ./data/Web Attack Brute Force.csv: 1507 samples
Epoch [4166/4167] Batch 24/24                 Loss D: -122802632.0000, Loss G: -0.1978
Training on ./data/Web Attack XSS.csv: 652 samples
Epoch [9090/9091] Batch 11/11                 Loss D: -121820224.0000, Loss G: -0.1703
Training on ./data/BENIGN.csv: 20000 samples
Epoch [319/320] Batch 313/313                 Loss D: -2.5565, Loss G: -0.02307
Training on ./data/Web_Attack_Sql_Injection.csv: 21 samples
Epoch [100000/100001] Batch 1/1                 Loss D: -115448440.0000, Loss G: -0.1656
Training on ./data/Web_Attack_XSS.csv: 652 samples
Epoch [9090/9091] Batch 11/11                 Loss D: -109647872.0000, Loss G: -0.2135
Training on ./data/DoS Slowhttpte

In [17]:
def generate_fake_data(generator, latent_dim, num_samples=1000):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(num_samples, latent_dim).to(device)
        fake_data = generator(z).cpu().numpy()
        fake_data = pd.DataFrame(fake_data, columns=dataset.features.columns)
    return fake_data

generated_samples = generate_fake_data(generator, LATENT_DIM, num_samples=1000)

def denormalize(data, min_values, max_values):
    # Assuming the original data is in the range [0, 1]
    # Adjust this function based on the original data's range
    return data * (max_values - min_values) + min_values

min_values = dataset.features.min().values
max_values = dataset.features.max().values

denormalize_data = denormalize(generated_samples, min_values, max_values)

denormalize_data.to_csv("./denormalized_fake_data.csv")

In [18]:
torch.save({
    'generator_state_dict': generator.state_dict(),
    'critic_state_dict': critic.state_dict(),
    'generator_optimizer_state_dict': generator_optimizer.state_dict(),
    'critic_optimizer_state_dict': critic_optimizer.state_dict(),
}, 'wgan_cicflow_model.pth')